In [10]:
import pdfplumber
import re
import pandas as pd
from collections import namedtuple

In [14]:
Line = namedtuple('Line','TEST RESULTS UNIT BIOLOGICAL_REF INTERVAL')

In [3]:
test_re = re.compile(r"(\w+(?:\s*\([\w\s]*\))?)\s*([HL]?)\s*(\d+\.\d+|\d+)\s*([^\d\s]+)\s*(\d+\.\d+)\s*-\s*(\d+\.\d+)")

In [9]:
test_re.search('Haemoglobin L 9.5 G% 13.00 - 17.00').group(1)

'Haemoglobin'

In [5]:
file = "bloodreport.pdf"

In [6]:
lines = []

with pdfplumber.open(file) as pdf:
    pages = pdf.pages[1:]
    for page in pages:
        text = page.extract_text()  
        for line in text.split('\n'):
            if isinstance(line, str):
                test = test_re.search(line)
                if test:
                    lines.append({
                        "Test Name": test.group(1),
                        "Status": test.group(2) if test.group(2) else "Normal",
                        "Value": test.group(3),
                        "Unit": test.group(4).strip(),
                        "Biological Reference Interval": f"{test.group(5)} - {test.group(6)}"
                    })
                    print(lines[-1])
            else:
                print("Non-string line encountered, skipping:", line)

{'Test Name': 'Haemoglobin', 'Status': 'L', 'Value': '9.5', 'Unit': 'G%', 'Biological Reference Interval': '13.00 - 17.00'}
{'Test Name': 'RBC (Electrical Impedance)', 'Status': 'L', 'Value': '3.55', 'Unit': 'millions/cumm', 'Biological Reference Interval': '4.50 - 5.50'}
{'Test Name': 'PCV(Calc)', 'Status': 'L', 'Value': '29.50', 'Unit': '%', 'Biological Reference Interval': '40.00 - 50.00'}
{'Test Name': 'MCV (RBC histogram)', 'Status': 'Normal', 'Value': '83.1', 'Unit': 'fL', 'Biological Reference Interval': '83.00 - 101.00'}
{'Test Name': 'MCH (Calc)', 'Status': 'L', 'Value': '26.7', 'Unit': 'pg', 'Biological Reference Interval': '27.00 - 32.00'}
{'Test Name': 'MCHC (Calc)', 'Status': 'Normal', 'Value': '32.1', 'Unit': 'gm/dL', 'Biological Reference Interval': '31.50 - 34.50'}
{'Test Name': 'RDW (RBC histogram)', 'Status': 'H', 'Value': '20.50', 'Unit': '%', 'Biological Reference Interval': '11.00 - 16.00'}
{'Test Name': 'Count', 'Status': 'Normal', 'Value': '5510', 'Unit': '/µL', 

In [7]:
df =pd.DataFrame(lines)
df

,Test Name,Status,Value,Unit,Biological Reference Interval
0,Haemoglobin,L,9.5,G%,13.00 - 17.00
1,RBC (Electrical Impedance),L,3.55,millions/cumm,4.50 - 5.50
2,PCV(Calc),L,29.50,%,40.00 - 50.00
3,MCV (RBC histogram),Normal,83.1,fL,83.00 - 101.00
4,MCH (Calc),L,26.7,pg,27.00 - 32.00
5,MCHC (Calc),Normal,32.1,gm/dL,31.50 - 34.50
6,RDW (RBC histogram),H,20.50,%,11.00 - 16.00
7,Count,Normal,5510,/µL,4000.00 - 10000.00
8,Neutrophil,Normal,65.0,%,40.00 - 70.00
9,Lymphocyte,Normal,24.0,%,20.00 - 40.00


In [8]:
df.to_csv("bloodreport.csv",index=False)